In [1]:
import pandas as pd
from pycaret.regression import *

df = pd.read_csv("dataset/99_ready.csv")

df = df.drop(columns=["Unnamed: 0", "name"], errors='ignore')
df


,price,location,bedroom,bathroom,land_area,building_area
0,200.0,Surabaya,2,1,78.0,78.0
1,200.0,"Jambangan, Surabaya",2,1,72.0,70.0
2,200.0,Surabaya,3,1,152.0,150.0
3,200.0,Surabaya,2,1,78.0,75.0
4,235.0,Surabaya,2,1,69.0,65.0
...,...,...,...,...,...,...
6753,2500.0,Surabaya,4,4,150.0,200.0
6754,2500.0,"Sukolilo, Surabaya",4,2,147.0,200.0
6755,2500.0,Surabaya,3,2,240.0,300.0
6756,2500.0,Surabaya,2,1,105.0,125.0


In [2]:
df.duplicated().sum()

1457

In [3]:
df = df.drop_duplicates()

In [4]:
df.duplicated().sum()

0

In [ ]:
# import ydata_profiling as pp

# profile = pp.ProfileReport(df)
# profile.to_file('pp_report/Report_99.html')

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

100%|██████████| 6/6 [00:00<00:00, 260.85it/s]


Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

In [9]:
setup(
    df, 
    target='price',
    transform_target=True, 
    normalize=True, 
    normalize_method='minmax',
    remove_outliers=True, 
    outliers_threshold=0.05,
    categorical_features=['location'],
    session_id=42
)

,Description,Value
0,Session id,42
1,Target,price
2,Target type,Regression
3,Original data shape,"(5301, 6)"
4,Transformed data shape,"(5115, 6)"
5,Transformed train set shape,"(3524, 6)"
6,Transformed test set shape,"(1591, 6)"
7,Numeric features,4
8,Categorical features,1
9,Preprocess,True


In [10]:
best = compare_models()

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
catboost,CatBoost Regressor,278.9185,137828.2583,370.9545,0.5922,0.2945,0.2382,0.6810
lightgbm,Light Gradient Boosting Machine,282.4740,141557.0329,375.9377,0.5808,0.2988,0.2428,0.1750
gbr,Gradient Boosting Regressor,286.9182,142468.7526,377.0518,0.5786,0.2991,0.2458,0.0670
rf,Random Forest Regressor,285.9733,149170.5674,385.8511,0.5587,0.3041,0.2436,0.2030
xgboost,Extreme Gradient Boosting,285.8727,150807.2688,387.9971,0.5541,0.3051,0.2419,0.0480
et,Extra Trees Regressor,292.3813,156106.3243,394.7163,0.5384,0.3083,0.2465,0.1080
ada,AdaBoost Regressor,318.6324,161660.3182,401.8925,0.5216,0.3280,0.2851,0.0450
knn,K Neighbors Regressor,312.3564,171249.8828,413.4809,0.4931,0.3245,0.2662,0.0960
dt,Decision Tree Regressor,353.5922,240853.2281,490.3875,0.2870,0.3861,0.2895,0.0820
en,Elastic Net,467.9561,310974.2285,557.5150,0.0797,0.4591,0.4515,0.0400


In [11]:
tuned_model = tune_model(best, search_library='scikit-optimize', search_algorithm='bayesian')

,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,268.7252,121508.7123,348.5810,0.6273,0.2730,0.2271
1,276.7708,136736.3102,369.7787,0.5753,0.2929,0.2296
2,267.7462,123338.5400,351.1959,0.6396,0.2558,0.2054
3,288.7957,144811.4697,380.5410,0.5656,0.2971,0.2447
4,297.6371,161246.3945,401.5550,0.5332,0.3297,0.2764
5,273.8475,138332.3624,371.9306,0.6283,0.3013,0.2358
6,290.3531,150666.7931,388.1582,0.5790,0.3184,0.2683
7,286.8460,140266.5530,374.5218,0.5709,0.3112,0.2553
8,272.0319,120435.0783,347.0376,0.6118,0.2764,0.2290


Fitting 10 folds for each of 1 candidates, totalling 10 fits
Fitting 10 folds for each of 1 candidates, totalling 10 fits
Fitting 10 folds for each of 1 candidates, totalling 10 fits
Fitting 10 folds for each of 1 candidates, totalling 10 fits
Fitting 10 folds for each of 1 candidates, totalling 10 fits
Fitting 10 folds for each of 1 candidates, totalling 10 fits
Fitting 10 folds for each of 1 candidates, totalling 10 fits
Fitting 10 folds for each of 1 candidates, totalling 10 fits
Fitting 10 folds for each of 1 candidates, totalling 10 fits
Fitting 10 folds for each of 1 candidates, totalling 10 fits
Original model was better than the tuned model, hence it will be returned. NOTE: The display metrics are for the tuned model (not the original one).


In [12]:
holdout_pred = predict_model(tuned_model)

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,CatBoost Regressor,273.7739,128166.8829,358.0040,0.6446,0.2914,0.2420


In [13]:
new_data = df.copy()
new_data.drop('price', axis=1, inplace=True)
new_data.head()

,location,bedroom,bathroom,land_area,building_area
0,Surabaya,2,1,78.0,78.0
1,"Jambangan, Surabaya",2,1,72.0,70.0
2,Surabaya,3,1,152.0,150.0
3,Surabaya,2,1,78.0,75.0
4,Surabaya,2,1,69.0,65.0


In [14]:
predictions = predict_model(tuned_model, data = new_data)
predictions.head()

,location,bedroom,bathroom,land_area,building_area,prediction_label
0,Surabaya,2,1,78.0,78.0,899.111061
1,"Jambangan, Surabaya",2,1,72.0,70.0,801.322390
2,Surabaya,3,1,152.0,150.0,1372.118454
3,Surabaya,2,1,78.0,75.0,879.821483
4,Surabaya,2,1,69.0,65.0,709.779140


In [15]:
save_model(tuned_model, '99-CB')

Transformation Pipeline and Model Successfully Saved


(Pipeline(memory=Memory(location=None),
          steps=[('target_transformation',
                  TransformerWrapperWithInverse(transformer=TargetTransformer(estimator=PowerTransformer(standardize=False)))),
                 ('numerical_imputer',
                  TransformerWrapper(include=['bedroom', 'bathroom', 'land_area',
                                              'building_area'],
                                     transformer=SimpleImputer())),
                 ('categorical_imputer',
                  TransformerWrapper(...
                 ('rest_encoding',
                  TransformerWrapper(include=['location'],
                                     transformer=TargetEncoder(cols=['location'],
                                                               handle_missing='return_nan'))),
                 ('remove_outliers',
                  TransformerWrapper(transformer=RemoveOutliers(random_state=42))),
                 ('normalize', TransformerWrapper(transformer=